In [ ]:
################################## KENNETH EKPETERE (PhD) ####################################
##################### MODIFIED/STANDARDIZED RAINFALL ANOMALY INDEX #########################
##################################### (C) 2024  ########################################

In [1]:
import time
import hydroeval as he
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
import sys
from datetime import date
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from scipy.stats import sem
from sklearn.metrics import mean_squared_error
import math
from scipy import stats
import statistics
import os

#### **Rainfall Anomaly Index (RAI)**

In [7]:
# Define file paths and folder names
input_folder = "Data"
output_folder = "Results"
input_file = os.path.join(input_folder, "matchedData.csv")
output_file = os.path.join(output_folder, "anomalyDataRAI.csv")

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the data
df = pd.read_csv(input_file)

# Filter data for the years 2001 to 2022
df_filtered = df[(df['Year'] >= 2001) & (df['Year'] <= 2022)]

# Define the function for calculating RAI
def calculate_rai(df, column):
    P_mean = df[column].mean()
    P_top10 = df[column].nlargest(10).mean()
    P_bottom10 = df[column].nsmallest(10).mean()
    
    def rai_value(P_n):
        if P_n > P_mean:
            return 3 * ((P_n - P_mean) / (P_top10 - P_mean))
        else:
            return -3 * ((P_n - P_mean) / (P_bottom10 - P_mean))
    
    # Clip the RAI values between -3 and +3
    return np.clip(df[column].apply(rai_value), -3, 3)

# Loop through unique IDs and calculate RAI
output_list = []
for unique_id in df_filtered['ID'].unique():
    df_id = df_filtered[df_filtered['ID'] == unique_id].copy()
    
    # Calculate RAI for obs24h and pre24h
    df_id['obs24h_RAI'] = calculate_rai(df_id, 'obs24h')
    df_id['pre24h_RAI'] = calculate_rai(df_id, 'pre24h')
    
    # Append results to the output list
    output_list.append(df_id)

# Combine the processed DataFrames
df_output = pd.concat(output_list)

# Save the results to a CSV file
df_output.to_csv(output_file, index=False)

print(f"RAI output saved to {output_file}")


RAI output saved to Results\anomalyDataRAI.csv


#### **Modified Rainfall Anomaly Index (MRAI)**

In [8]:
# Define file paths and folder names
input_folder = "Data"
output_folder = "Results"
input_file = os.path.join(input_folder, "matchedData.csv")
output_file = os.path.join(output_folder, "anomalyMRAI.csv")

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the data
df = pd.read_csv(input_file)

# Filter data for the years 2001 to 2022
df_filtered = df[(df['Year'] >= 2001) & (df['Year'] <= 2022)]

# Define the function for calculating MRAI
def calculate_mrai(df, column):
    P_mean = df[column].mean()
    P_max = df[column].max()
    P_min = df[column].min()
    
    def mrai_value(P_n):
        if P_n > P_mean:
            return 3 * ((P_n - P_mean) / (P_max - P_mean))
        else:
            return -3 * ((P_n - P_mean) / (P_min - P_mean))
    
    return df[column].apply(mrai_value)

# Loop through unique IDs and calculate MRAI
output_list = []
for unique_id in df_filtered['ID'].unique():
    df_id = df_filtered[df_filtered['ID'] == unique_id].copy()
    
    # Calculate MRAI for obs24h and pre24h
    df_id['obs24h_MRAI'] = calculate_mrai(df_id, 'obs24h')
    df_id['pre24h_MRAI'] = calculate_mrai(df_id, 'pre24h')
    
    # Append results to the output list
    output_list.append(df_id)

# Combine the processed DataFrames
df_output = pd.concat(output_list)

# Save the results to a CSV file
df_output.to_csv(output_file, index=False)

print(f"MRAI output saved to {output_file}")


MRAI output saved to Results\anomalyMRAI.csv


#### **Standardized Rainfall Anomaly Index (SRAI)**

In [9]:
# Define file paths and folder names
input_folder = "Data"
output_folder = "Results"
input_file = os.path.join(input_folder, "matchedData.csv")
output_file = os.path.join(output_folder, "anomalySRAI.csv")

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the data
df = pd.read_csv(input_file)

# Filter data for the years 2001 to 2022
df_filtered = df[(df['Year'] >= 2001) & (df['Year'] <= 2022)]

# Define the function for calculating SRAI
def calculate_srai(df, column):
    P_mean = df[column].mean()
    P_std = df[column].std()
    
    def srai_value(P_n):
        if P_n > P_mean:
            return 3 * ((P_n - P_mean) / P_std)
        else:
            return -3 * ((P_n - P_mean) / P_std)
    
    # Clip the SRAI values between -3 and +3
    return np.clip(df[column].apply(srai_value), -3, 3)

# Loop through unique IDs and calculate SRAI
output_list = []
for unique_id in df_filtered['ID'].unique():
    df_id = df_filtered[df_filtered['ID'] == unique_id].copy()
    
    # Calculate SRAI for obs24h and pre24h
    df_id['obs24h_SRAI'] = calculate_srai(df_id, 'obs24h')
    df_id['pre24h_SRAI'] = calculate_srai(df_id, 'pre24h')
    
    # Append results to the output list
    output_list.append(df_id)

# Combine the processed DataFrames
df_output = pd.concat(output_list)

# Save the results to a CSV file
df_output.to_csv(output_file, index=False)

print(f"SRAI output saved to {output_file}")


SRAI output saved to Results\anomalySRAI.csv


#### **Combined Anomaly Index**

In [10]:
# Define file paths and folder names
input_folder = "Data"
output_folder = "Results"
input_file = os.path.join(input_folder, "matchedData.csv")
output_file = os.path.join(output_folder, "anomalyAll.csv")

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the data
df = pd.read_csv(input_file)

# Filter data for the years 2001 to 2022
df_filtered = df[(df['Year'] >= 2001) & (df['Year'] <= 2022)]

# Define functions for calculating RAI, MRAI, and SRAI

def calculate_rai(df, column):
    P_mean = df[column].mean()
    P_top10 = df[column].nlargest(10).mean()
    P_bottom10 = df[column].nsmallest(10).mean()
    
    def rai_value(P_n):
        if P_n > P_mean:
            return 3 * ((P_n - P_mean) / (P_top10 - P_mean))
        else:
            return -3 * ((P_n - P_mean) / (P_bottom10 - P_mean))
    
    # Clip the RAI values between -3 and +3
    return np.clip(df[column].apply(rai_value), -3, 3)

def calculate_mrai(df, column):
    P_mean = df[column].mean()
    P_max = df[column].max()
    P_min = df[column].min()
    
    def mrai_value(P_n):
        if P_n > P_mean:
            return 3 * ((P_n - P_mean) / (P_max - P_mean))
        else:
            return -3 * ((P_n - P_mean) / (P_min - P_mean))
    
    # MRAI values are already confined to the range -3 and +3, so no need to clip
    return df[column].apply(mrai_value)

def calculate_srai(df, column):
    P_mean = df[column].mean()
    P_std = df[column].std()
    P_max = df[column].max()
    P_min = df[column].min()
    
    def srai_value(P_n):
        if P_n > P_mean:
            return 3 * ((P_n - P_mean) / (P_max - P_std))
        else:
            return -3 * ((P_n - P_mean) / (P_min - P_std))
    
    # Clip the SRAI values between -3 and +3
    return np.clip(df[column].apply(srai_value), -3, 3)

# Loop through unique IDs and calculate RAI, MRAI, and SRAI
output_list = []
for unique_id in df_filtered['ID'].unique():
    df_id = df_filtered[df_filtered['ID'] == unique_id].copy()
    
    # Calculate RAI, MRAI, SRAI for obs24h
    df_id['obs24h_RAI'] = calculate_rai(df_id, 'obs24h')
    df_id['obs24h_MRAI'] = calculate_mrai(df_id, 'obs24h')
    df_id['obs24h_SRAI'] = calculate_srai(df_id, 'obs24h')
    
    # Calculate RAI, MRAI, SRAI for pre24h
    df_id['pre24h_RAI'] = calculate_rai(df_id, 'pre24h')
    df_id['pre24h_MRAI'] = calculate_mrai(df_id, 'pre24h')
    df_id['pre24h_SRAI'] = calculate_srai(df_id, 'pre24h')
    
    # Append results to the output list
    output_list.append(df_id)

# Combine the processed DataFrames
df_output = pd.concat(output_list)

# Save the results to a CSV file
df_output.to_csv(output_file, index=False)

print(f"Output saved to {output_file}")


Output saved to Results\anomalyAll.csv
